# 

# Métrica 4_1_09 Coincidencia entre nombre de empresa donde participa y nombre/razón social en ingresos
| Escenario                                                                     | Condición                                                                             | Resultado        |
| ----------------------------------------------------------------------------- | ------------------------------------------------------------------------------------- | ---------------- |
| No aplica                                                                     | La participación **no** está declarada como “Declarante” **o** no recibe remuneración | ⚪ **NO_APLICA**  |
| Falta nombre en intereses o en ingresos                                       | Nombre en DI o DSP está vacío o nulo                                                  | ⚪ **SIN_DATO**   |
| El nombre de empresa en intereses NO coincide con la razón social en ingresos | Strings diferentes (normalizados)                                                     | 🔴 **NO_CUMPLE** |
| El nombre de empresa coincide                                                 | Ambos nombres coinciden (insensible a may./min., espacios, acentos)                   | 🟢 **CUMPLE**    |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata

METRIC_ID = "4_1_09_CONGRUENCIA_NOMBRE_EMPRESA"

# -------------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------------

def extraer_valor(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None

def normalizar_texto(v):
    if not v:
        return None
    v = str(v).strip().upper()
    # remover acentos
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    # quitar dobles espacios
    while "  " in v:
        v = v.replace("  ", " ")
    return v


# -------------------------------------------------------------------
# EVALUADOR
# -------------------------------------------------------------------

def evaluar_metrica(doc):

    # DI - Participación
    recibe = extraer_valor(doc, "declaracion.intereses.participacion.participacion.recibeRemuneracion")
    tipo_part = normalizar_texto(extraer_valor(doc, "declaracion.intereses.participacion.participacion.tipo"))
    nombre_di = normalizar_texto(extraer_valor(doc, "declaracion.intereses.participacion.participacion.nombreEmpresa"))

    # DSP - Ingresos II.1
    nombre_ing = normalizar_texto(
        extraer_valor(doc,
            "declaracion.situacionPatrimonial.ingresos."
            "actividadIndustialComercialEmpresarial.actividades.nombreRazonSocial"
        )
    )

    # 1. NO APLICA: no es el declarante o no recibe remuneración
    if tipo_part != "DECLARANTE" or str(recibe).upper() != "SI":
        return "NO_APLICA"

    # 2. SIN DATO: falta cualquiera de los nombres
    if not nombre_di or not nombre_ing:
        return "SIN_DATO"

    # 3. CUMPLE / NO_CUMPLE según coincidencia
    if nombre_di == nombre_ing:
        return "CUMPLE"

    return "NO_CUMPLE"


# -------------------------------------------------------------------
# PROCESAMIENTO MONGO
# -------------------------------------------------------------------

def procesar_metrica_4_1_09():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.participacion": 1,
            "declaracion.situacionPatrimonial.ingresos": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"> {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_09()
